In [ ]:
# Heart Disease Prediction

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# Read in data and treat to reduce categorical size
data = pd.read_csv("heart_2020_original.csv")

# Functions to reduce category sizes
def disDiabetes(data):
    diab = (data["Diabetic"])
    if diab[0] == "Y":
        return "Yes"
    else:
        return "No"

def disAge(data):
    age = data["AgeCategory"]
    if age == "18-24" or age == "25-29" or age == "30-34" or age == "35-39":
        return "18-39"
    elif age == "40-44" or age == "45-49" or age == "50-54" or age == "55-59":
        return "40-59"
    else:
        return "60 or older"

def treat(data):
    # Remove unused columns
    data.drop(["SkinCancer"], axis=1, inplace=True)
    data.drop(["Race"], axis=1, inplace=True)

    # Adjust value representation
    data["Diabetic"] = data.apply(disDiabetes, axis=1)
    data["AgeCategory"] = data.apply(disAge, axis=1)

treat(data)

In [ ]:
# Split data for training and testing
train_data = data.sample(frac=0.75, random_state=21)
test_data = data.drop(train_data.index)

# Get heart disease distribution
data["HeartDisease"].value_counts(normalize=True)

# Only 8.5% target heart disease so use undersampling for a better model
rus = RandomUnderSampler(random_state=21)
X_train_data_us, Y_train_data_us = rus.fit_resample(train_data.drop(["HeartDisease"], axis=1), train_data["HeartDisease"])

# Merge undersampled data for ease of reading with CSV
train_data_us = pd.concat([Y_train_data_us, X_train_data_us], axis=1)

# Output CSVs for loading in BN
train_data_us.to_csv("heart_train.csv", index=False)
#train_data.to_csv("heart_train.csv", index=False)
test_data.to_csv("heart_test.csv", index=False)

## EDA

In [ ]:
import phik
from phik.report import plot_correlation_matrix
from phik import report

phik_overview = train_data_us.phik_matrix()
phik_overview.round(1)
plot_correlation_matrix(phik_overview.values,
                        x_labels=phik_overview.columns,
                        y_labels=phik_overview.index,
                        vmin=0, vmax=1,
                        title=r"correlation $\phi_K$",
                        fontsize_factor=0.5,
                        figsize=(10, 8))
plt.tight_layout()
plt.savefig("corr.png")

NameError: name 'train_data_us' is not defined

In [ ]:
# Define BN topology
bn = gum.BayesNet("Heart Disease Prediction")

bn.add(gum.LabelizedVariable("HeartDisease", "HeartDisease", ["Yes", "No"]))
bn.add(gum.DiscretizedVariable("BMI", "BMI").addTick(0).addTick(18.5).addTick(25).addTick(30).addTick(100))
bn.add(gum.LabelizedVariable("Smoking", "Smoking", ["Yes", "No"]))
bn.add(gum.LabelizedVariable("AlcoholDrinking",
       "AlcoholDrinking", ["Yes", "No"]))
bn.add(gum.LabelizedVariable("Stroke", "Stroke", ["Yes", "No"]))
bn.add(gum.DiscretizedVariable("PhysicalHealth", "PhysicalHealth").addTick(0).addTick(1).addTick(10).addTick(30))
bn.add(gum.DiscretizedVariable("MentalHealth", "MentalHealth").addTick(
    0).addTick(1).addTick(10).addTick(30))
bn.add(gum.LabelizedVariable("DiffWalking", "DiffWalking", ["Yes", "No"]))
bn.add(gum.LabelizedVariable("Sex", "Sex", ["Male", "Female"]))
bn.add(gum.LabelizedVariable("AgeCategory", "AgeCategory", [
       "18-39", "40-59", "60 or older"]))
bn.add(gum.LabelizedVariable("Diabetic", "Diabetic", ["Yes", "No"]))
bn.add(gum.LabelizedVariable("PhysicalActivity", "PhysicalActivity", ["Yes", "No"]))
bn.add(gum.LabelizedVariable("GenHealth", "GenHealth", ["Very good", "Good", "Excellent", "Fair", "Poor"]))
bn.add(gum.DiscretizedVariable("SleepTime", "SleepTime").addTick(0).addTick(5).addTick(7).addTick(9).addTick(24))
bn.add(gum.LabelizedVariable("Asthma", "Asthma", ["Yes", "No"]))
bn.add(gum.LabelizedVariable("KidneyDisease", "KidneyDisease", ["Yes", "No"]))


In [ ]:
# View nodes added
print(bn.variable("HeartDisease"))
print(bn.variable("BMI"))
print(bn.variable("Smoking"))
print(bn.variable("AlcoholDrinking"))
print(bn.variable("Stroke"))
print(bn.variable("PhysicalHealth"))
print(bn.variable("MentalHealth"))
print(bn.variable("DiffWalking"))
print(bn.variable("Sex"))
print(bn.variable("AgeCategory"))
print(bn.variable("Diabetic"))
print(bn.variable("PhysicalActivity"))
print(bn.variable("GenHealth"))
print(bn.variable("SleepTime"))
print(bn.variable("Asthma"))
print(bn.variable("KidneyDisease"))

bn

In [ ]:
# Define the BN structure

links = [("PhysicalActivity","HeartDisease"),
         ("AgeCategory","PhysicalActivity"),
         ("AgeCategory","HeartDisease"),
         ("BMI", "PhysicalActivity"),
         ("BMI", "Diabetic"),
         ("GenHealth", "HeartDisease"),
         ("PhysicalHealth", "GenHealth"),
         ("MentalHealth", "GenHealth"),
         ("SleepTime", "GenHealth"),
         ("BMI", "GenHealth"),
         ("Sex","HeartDisease"),
         ("Smoking","HeartDisease"),
         ("AlcoholDrinking","HeartDisease"),
         ("Diabetic","KidneyDisease"),
         ("KidneyDisease","HeartDisease"),
         ("Diabetic","HeartDisease"),
         ("HeartDisease","DiffWalking"),
         ("HeartDisease", "Stroke"),
         ("Asthma","DiffWalking")]

for link in links:
    bn.addArc(*link)

In [ ]:
# Visualise the network with arcs
bn

In [ ]:
# Learn the parameters of the BN
learner = gum.BNLearner("heart_train.csv", bn)
learner.useSmoothingPrior(2)
bn = learner.learnParameters(bn.dag())
# Save Model
gum.saveBN(bn, "out2.bifxml")

In [ ]:
# View learned model
gnb.showInference(bn)
print(bn.cpt("HeartDisease"))

In [ ]:
# Validate model accuracy

from pyAgrum.lib.bn2roc import showROC_PR
import pyAgrum.lib.explain as explain

explain.showInformation(bn)

showROC_PR(bn, "heart_train.csv", "HeartDisease", "Yes", False, True)
showROC_PR(bn, "heart_test.csv", "HeartDisease", "Yes", False, True)

# Decision Network
## Set up
Set up Topology and Weights of Decision Network using previous Bayesian Network

In [ ]:
# Set up Decision Network
dn = gum.loadID("out2.bifxml")
dn.addUtilityNode(gum.LabelizedVariable("U", "U", 1))
dn.addArc("HeartDisease", "U")
decisionNodes = ["DecreaseSubstanceUse", "GoToDoctor", "IncreasePhysicalActivity"]
for node in decisionNodes:
    dn.addDecisionNode(gum.LabelizedVariable(node, node, ["Yes", "No"]))
    dn.addArc(node, "U")
dn

## Define Utility Function

In [ ]:
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "Yes"}] = 100
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "No"}] = 66
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "No", "IncreasePhysicalActivity": "Yes"}] = 66
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "No", "IncreasePhysicalActivity": "No"}] = 33
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "No", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "Yes"}] = 33
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "No", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "No"}] = 33
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "No", "GoToDoctor": "No", "IncreasePhysicalActivity": "Yes"}] = 33
dn.utility("U")[{"HeartDisease": "Yes", "DecreaseSubstanceUse": "No", "GoToDoctor": "No", "IncreasePhysicalActivity": "No"}] = 0
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "Yes"}] = 25
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "No"}] = 33
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "No", "IncreasePhysicalActivity": "Yes"}] = 33
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "Yes", "GoToDoctor": "No", "IncreasePhysicalActivity": "No"}] = 16
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "No", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "Yes"}] = 16
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "No", "GoToDoctor": "Yes", "IncreasePhysicalActivity": "No"}] = 16
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "No", "GoToDoctor": "No", "IncreasePhysicalActivity": "Yes"}] = 16
dn.utility("U")[{"HeartDisease": "No", "DecreaseSubstanceUse": "No", "GoToDoctor": "No", "IncreasePhysicalActivity": "No"}] = 0

In [ ]:
ie=gum.ShaferShenoyLIMIDInference(dn)
try:
    ie.makeInference()
except gum.GumException as e:
    print(e)

ie.addNoForgettingAssumption(["DecreaseSubstanceUse", "GoToDoctor", "IncreasePhysicalActivity"])
gnb.sideBySide(gnb.getInference(dn,engine=ie, evs={'DecreaseSubstanceUse': 'No'}))

# Model Testing:
## Initial Probability Graph

In [ ]:
ie = gum.VariableElimination(bn)
gnb.showInference(bn, engine=ie)

## Examples of Diagnostic Reasoning
1) What is the probability that someone is over 60 given that they have Heart Disease

In [ ]:
ie.setEvidence({"HeartDisease": "Yes"})
ie.makeInference()
print(ie.posterior("AgeCategory")[2])
gnb.showPosterior(bn, evs={"HeartDisease": "Yes"}, target="AgeCategory")
#print(gnb.showInference(bn, evs={"AgeCategory": "60 or older"}, engine=ie))

2) What is the probability that someone is male given they do not have heart disease and are diabetic

In [ ]:
ie.setEvidence({"HeartDisease": "Yes", "Diabetic": "Yes"})
ie.makeInference()
print(ie.posterior("Sex")[0])
gnb.showPosterior(bn, evs={"HeartDisease": "Yes", "Diabetic": "Yes"}, target="Sex")

## Example of Casual Reasoning
1) Probability that someone has heart disease given they are heavy drinkers with kidney disease

In [ ]:
ie.setEvidence({"KidneyDisease": "Yes", "AlcoholDrinking" : "Yes"})
ie.makeInference
print(ie.posterior("HeartDisease")[0])
gnb.showPosterior(bn, {"KidneyDisease": "Yes", "AlcoholDrinking" : "Yes"}, target="HeartDisease")

2) Probability that someone has heart disease given that they are over 60 and have difficulty walking

In [ ]:
ie.setEvidence({"AgeCategory": "60 or older", "DiffWalking" : "Yes"})
ie.makeInference()
print(ie.posterior("HeartDisease")[0])
gnb.showPosterior(bn, {"AgeCategory": "60 or older", "DiffWalking" : "Yes"}, target="HeartDisease")

3) Probability that someone has heart disease given that they are 18-39 and do not smoke

In [ ]:
ie.setEvidence({"AgeCategory": "18-39", "Smoking" : "No"})
ie.makeInference()
print(ie.posterior("HeartDisease")[0])
gnb.showPosterior(bn, {"AgeCategory": "18-39", "Smoking" : "No"}, target="HeartDisease")

## Decision Networks
1) Utility of someone to stop drinking and smoking and go to the doctor given that are over 60 and have difficulty walking

In [ ]:
ie=gum.ShaferShenoyLIMIDInference(dn)
try:
    ie.makeInference()
except gum.GumException as e:
    print(e)

ie.addNoForgettingAssumption(["DecreaseSubstanceUse", "GoToDoctor", "IncreasePhysicalActivity"])
#gnb.show(dn.utility("U"))
gnb.sideBySide(gnb.getInference(dn,engine=ie, evs={'DecreaseSubstanceUse': 'Yes', "GoToDoctor": "Yes", "AgeCategory": "60 or older", "DiffWalking" : "Yes"}))

2)Utility of some to increase their physical activity given that they are not diabetic and have excellent general health

In [ ]:
gnb.sideBySide(gnb.getInference(dn,engine=ie, evs={'IncreasePhysicalActivity': 'Yes',"GenHealth": "Excellent", "Diabetic" : "No"}))